In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!nvidia-smi

Thu Apr  6 17:58:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:10:1C.0 Off |                    0 |
| N/A   48C    P0    58W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import sys
import os

sys.path.append(os.path.abspath("/data/home/ngjhn/read"))


In [4]:
from coco import sweep_iteration, Config, sweep
from adapter_utils import FTConf, AdapterConf

/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
!unset $LD_LIBRARY_PATH

In [6]:
sweep_config = {
    "method": "bayes",
    "name": "RNN 64",
    "metric": {
        "name": "val_loss",
        "goal": "maximize"
    },
    "parameters": {
        "learning_rate": {
            "min": 1e-6,
            "max": 1e-3
        },
    },
}
sweep(sweep_config)

wandb: Agent Starting Run: wlmtsd51 with config:
wandb: 	learning_rate: 1.603897716927574e-05


Starting sweep iteration


wandb: Currently logged in as: ngjhn (read). Use `wandb login --relogin` to force relogin


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/loggers/wandb.py:395: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertLMHeadModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertLMHeadModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertLMHeadModel were n

Enables rnn-adapter tuning; num trainable params 423,168, ratio 0.002
loading annotations into memory...
Done (t=0.82s)
creating index...
index created!
loading annotations into memory...
Done (t=0.38s)
creating index...
index created!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /data/home/ngjhn/.local/lib/python3.8/site-packages/ ...
  rank_zero_warn(


loading annotations into memory...
Done (t=0.75s)
creating index...
index created!
loading annotations into memory...
Done (t=0.35s)
creating index...
index created!


/data/home/ngjhn/miniconda3/envs/awsnccl/lib/python3.8/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /data/home/ngjhn/read/lightning_logs/rnn exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")

  | Name      | Type                      | Params
--------------------------------------------------------
0 | model     | VisionEncoderDecoderModel | 224 M 
1 | val_loss  | AvgMetric                 | 0     
2 | test_loss | AvgMetric                 | 0     
--------------------------------------------------------
423 K     Trainable params
224 M     Non-trainable params
224 M     Total params
898.774   Total estimated model params size (MB)


Epoch 0:   0%|          | 0/2587 [00:00<?, ?it/s] 